# 2. Preprocessing EXTOD education dataset

The goal of this notebook is to prepare the EXTOD education data for machine learning - predicting euglycemia during and around exercise.

EXTOD education was a pilot study in which 106 participants were randomly allocated to either standard care or an education programme teaching them how better to manage blood glucose around exercise.

The data used will be demographic, lab, clinical and physiological if available.

### Objectives:
1. Clean and combine the exercise diaries
2. Preprocess demographic data
3. Preprcoess lab data
4. Prepare CGM data

## 2.0. Import packages

In [1]:
# Import packages and upload dataset
import pandas as pd
import numpy as np
import datetime
from datetime import datetime as dt
import os
import math
from datetime import timedelta as time
import warnings
import preprocess_helper
warnings.filterwarnings('ignore')

In [2]:
diary_directory = '../../data/raw_data/extod_edu/diary_data/'
cgm_directory = '../../data/raw_data/extod_edu/cgm_data/'

## 2.1. Clean and combine exercise diaries

### 2.1.1. Load files and rename columns

In [3]:
ex_diary_tn_bsl = pd.read_excel(diary_directory+'taunton.xlsx', sheet_name='Taunton_baseline')
ex_diary_tn_6m = pd.read_excel(diary_directory+'taunton.xlsx', sheet_name='Taunton_6months')
ex_diary_bm_bsl = pd.read_excel(diary_directory+'birmingham.xlsx',  sheet_name='Birmingham_baseline')
ex_diary_bm_6m = pd.read_excel(diary_directory+'birmingham.xlsx',  sheet_name='Birmingham_6months')

In [4]:
# Create list with all diaries in
diaries = [ex_diary_tn_bsl, ex_diary_tn_6m, ex_diary_bm_bsl, ex_diary_bm_6m]

In [5]:
# This diary has an extra column 'day'
ex_diary_bm_6m.drop(columns='Day', inplace=True)

In [6]:
# Clean diaries
diaries = [preprocess_helper.clean(i) for i in diaries]
# Concatenate them into one df
exercise_diaries = pd.concat(diaries, axis=0)
# Only select days where exercise has occurred
exercise_diaries = exercise_diaries.loc[(exercise_diaries.exercise_on_day=='Yes') | (exercise_diaries.exercise_on_day=='yes')]

In [7]:
size1 = exercise_diaries.shape[0]

In [8]:
size1

1017

In [9]:
len(exercise_diaries.ID.unique())

91

### 2.1.2. Clean datetime column

In [9]:
# Fix the finish time
exercise_diaries.finish_time = exercise_diaries.finish_time.apply(lambda x: preprocess_helper.check_time(x))

In [10]:
# Fix the start time
exercise_diaries.start_time = exercise_diaries.start_time.apply(lambda x: preprocess_helper.check_time(x))

In [11]:
# Combine date and time to make datetime for start and finish
exercise_diaries['start_datetime'] = exercise_diaries.apply(lambda row:
                                        datetime.datetime.combine(row.date,
                                       row.start_time) if pd.notnull(row.date)
                                        and pd.notnull(row.start_time) else 
                                       np.nan, axis=1)
exercise_diaries['finish_datetime'] = exercise_diaries.apply(lambda row: 
                                      datetime.datetime.combine(row.date,
                                      row.finish_time) if pd.notnull(row.date) 
                                      and pd.notnull(row.finish_time) else
                                                             np.nan, axis=1)

In [12]:
# 41 dropped with no datetime data
exercise_diaries_complete = exercise_diaries[(pd.notnull(exercise_diaries.duration_mins) &
                               (pd.notnull(exercise_diaries.start_datetime) |
                 pd.notnull(exercise_diaries.finish_datetime))) | (pd.notnull(exercise_diaries.start_datetime) &
                 pd.notnull(exercise_diaries.finish_datetime))]

In [13]:
# Fill gaps in datetime using other 2 measurements
exercise_diaries_complete = exercise_diaries_complete.apply(lambda row: preprocess_helper.fill_missing(row),
                                                            axis=1)

In [14]:
# Calculate duration from start and finish datetimes
#exercise_diaries_complete['duration_calc'] = exercise_diaries_complete.finish_datetime - exercise_diaries_complete.start_datetime
# Convert to minutes
exercise_diaries_complete['duration'] = (exercise_diaries_complete.finish_datetime - exercise_diaries_complete.start_datetime).apply(lambda x: datetime.timedelta.total_seconds(x)/60)

In [15]:
# Reset index
exercise_diaries_complete.reset_index(drop=True, inplace=True)

In [16]:
# Manually correct some of the incorrect datetimes 
exercise_diaries_complete.at[433, 'finish_datetime'] = pd.to_datetime('2018-09-12 17:30:00')
exercise_diaries_complete.at[457, 'finish_datetime'] = pd.to_datetime('2019-01-25 21:20:00')
exercise_diaries_complete.at[759, 'finish_datetime'] = pd.to_datetime('2018-04-16 19:30:00')
exercise_diaries_complete.at[589, 'finish_datetime'] = pd.to_datetime('2019-01-12 22:45:00')

In [17]:
# Manually switch start and finish time for those who put them in the wrong way
start_times = exercise_diaries_complete.finish_datetime.iloc[[760, 730, 642, 478]]
end_times = exercise_diaries_complete.start_datetime.iloc[[760, 730, 642, 478]] 
exercise_diaries_complete.start_datetime.iloc[[760, 730, 642, 478]] = start_times
exercise_diaries_complete.finish_datetime.iloc[[760, 730, 642, 478]] = end_times

In [18]:
# Drop and instances where duration is negative
exercise_diaries_complete.drop(exercise_diaries_complete.loc[exercise_diaries_complete['duration'] < 0].index, 
                               inplace=True)

In [19]:
# Drop unneeded columns 
exdi = exercise_diaries_complete.drop(columns=['date', 'exercise_on_day', 'start_time',
                          'finish_time', 'hours_in_mins', 'mins', #'duration_calc',
                          'duration_mins', 'comment'])

In [20]:
size2 = exdi.shape[0]

### 2.1.3. Sort out dates

In [21]:
# Make sure there's no minus durations
exdi = exdi.loc[exdi['duration']>0]

In [22]:
# Collec date information from start_datetime
exdi = preprocess_helper.date_preprocessing(exdi, 'start_datetime', False, True, True)

In [23]:
exdi.shape[0]

966

In [24]:
def handle_overlaps(group):
    # Sort by start_datetime
    group = group.sort_values(by='start_datetime')
    
    non_overlapping = []
    overlap_count = 0  # Counter to limit the number of printed overlaps
    for _, row in group.iterrows():
        if non_overlapping:
            last_interval = non_overlapping[-1]
            # Check for overlap
            if row['start_datetime'] < last_interval['finish_datetime']:
                # Print the overlapping intervals (limited to a few for brevity)
                if overlap_count < 5:
                    print(f"Overlap detected for ID {group['ID'].iloc[0]}:")
                    print(f"Interval 1: {last_interval['start_datetime']} to {last_interval['finish_datetime']}")
                    print(f"Interval 2: {row['start_datetime']} to {row['finish_datetime']}\n")
                    overlap_count += 1

                curr_duration = (row['finish_datetime'] - row['start_datetime']).seconds / 60
                last_duration = (last_interval['finish_datetime'] - last_interval['start_datetime']).seconds / 60
                
                # Decide which interval to keep
                if (curr_duration >= 10 and curr_duration <= 120) and (last_duration < 10 or last_duration > 120):
                    non_overlapping[-1] = row
                elif (last_duration >= 10 and last_duration <= 120) and (curr_duration < 10 or curr_duration > 120):
                    continue
                elif curr_duration > last_duration:
                    non_overlapping[-1] = row
            else:
                non_overlapping.append(row)
        else:
            non_overlapping.append(row)
    
    return pd.DataFrame(non_overlapping)

# Apply the function to each group
exdi = exdi.groupby('ID').apply(handle_overlaps).reset_index(drop=True)


Overlap detected for ID 1021:
Interval 1: 2018-02-21 11:22:00 to 2018-02-21 11:54:00
Interval 2: 2018-02-21 11:32:00 to 2018-02-21 12:55:00

Overlap detected for ID 2021:
Interval 1: 2018-04-26 14:15:00 to 2018-04-26 16:00:00
Interval 2: 2018-04-26 15:15:00 to 2018-04-26 15:30:00

Overlap detected for ID 2026:
Interval 1: 2018-03-13 20:30:00 to 2018-03-13 22:15:00
Interval 2: 2018-03-13 21:00:00 to 2018-03-13 22:00:00

Overlap detected for ID 2040:
Interval 1: 2019-02-02 20:50:00 to 2019-02-02 21:05:00
Interval 2: 2019-02-02 21:00:00 to 2019-02-02 21:20:00



In [25]:
exdi.shape[0]

962

### 2.1.4. Borg/intensity

In [26]:
# Reset index
exdi.reset_index(drop=True, inplace=True)

In [27]:
# Manually correct some of the very strangely entered borg scores
swap_dict = {'11\\12+ 14': 12.5, '16 for 30mins,12 after ': 14, '5, 11-13': 12}
exdi.borg = exdi.borg.replace(swap_dict)

In [28]:
# Correct borg score
exdi.borg = exdi.borg.apply(lambda x: preprocess_helper.correct_borg(x))

Commencing phase 2


In [29]:
# Rename to intensity
exdi.rename(columns={'borg':'intensity'}, inplace=True)

In [30]:
# Convert to 0-2 format
exdi['intensity'] = exdi['intensity'].apply(lambda x: preprocess_helper.replace_borg(x))

### 2.1.5. Type of exercise

In [31]:
# Convert form_of_exercise
exdi['form_of_exercise'] = exdi.type_of_exercise.apply(lambda x: 
                                                       preprocess_helper.divide_exercise_into_type(x))
# Drop any nulls
#exdi.dropna(subset=['form_of_exercise'], inplace=True)

gardening
gardening
gardening
gardening
gardening
manual work
manual work
manual work
manual work
gardening
gardening
hoovering
gardening
gardening
moving fence panels
singing
gardening
gardening
gardening
heavy farm work
gardening
garden & farming
gardening
garden & farming
farmwork
gardening
nk (teaching?)
gardening
farm work
gardening
gardening
gardening
gardening
garden & farming
gardening
farm work
gardening
gardening
hoovering
gardening
gardening
bathroom deep clean
gardening
shopping
gardening
hoovering
house move
gardening
lambing
lambing
lambing
sheep weighing
lambing
floor scrubbing
herding sheep
housework
farm work
housework
farm work
farm work
farm work
farm work
building work
gardening 
house work
work(kitchen)
work
housework
house work
gardening
gardening
gardening
gardening
gardening
drumming
kids play centre
gardening
diy
diy
diy
sax playing at band
sax playing at band
sax playing
gardening
gardening
gardening
housework
catching buses
band rehersal 
house work


In [32]:
exdi.shape[0]

962

## 2.2. Clean and combine CGM files

In [33]:
# Map the function to all files in the directory
results = list(map(lambda filename: preprocess_helper.format_df(filename, cgm_directory),
                   os.listdir(cgm_directory)))
# Concatenate all files to make one dataframe
df_total = pd.concat(results).reset_index(drop=True)

In [34]:
# Divide the label into id and period
df_total[['ID', 'period']] = df_total.ID.str.split('_', expand=True)

# Select only first part of id
df_total.ID.apply(lambda x: int(x[:4]))

0         2012
1         2012
2         2012
3         2012
4         2012
          ... 
235831    2045
235832    2045
235833    2045
235834    2045
235835    2045
Name: ID, Length: 235836, dtype: int64

## 2.3. Lab & demographic data

In [35]:
# Load files
demographics = pd.read_excel('../../data/raw_data/extod_edu/demographics.xlsx', sheet_name=1)
c_pep = pd.read_excel('../../data/raw_data/extod_edu/c_pep.xlsx')

In [36]:
# Set id as first 4 characters
demographics['ID'] = demographics.label.apply(lambda x: int(x[3:7]))

In [37]:
# Merge demographics and cpep on id
demo_lab = pd.merge(demographics, c_pep[['ID', 'Cpeptide (pmol/L)']], on='ID')

In [38]:
# Calculate bmi from weight and height
demo_lab['bmi'] = demo_lab.weight_vst1/(demo_lab.height_vst1/100 * demo_lab.height_vst1/100)

In [39]:
# Select useful columns
demo_lab = demo_lab[['ID', 'agedv', 'gender', 'mh_t1dm_duration_vst1', 'bmi',
                     'hba1c_result_vst1', 'Cpeptide (pmol/L)']]
# Rename columns
demo_lab.columns = ['ID', 'age', 'sex', 'years_since_diagnosis', 'bmi', 'hba1c',
                    'cpep']

In [40]:
# Set sex to lower case
demo_lab.sex = demo_lab.sex.apply(lambda x: x.lower())

In [41]:
# Replace cpep below 3 with 3
demo_lab.cpep.replace({'<3':2.9}, inplace=True)

In [42]:
# Set insulin modality for mdi for all
demo_lab['insulin_modality'] = 'mdi'

## 2.4. Save files

In [43]:
# Add 'helm' to IDs to make them unique
for i in [exdi, df_total, demo_lab]:
    i['ID'] = i['ID'].apply(lambda row: 'ext_edu_'+str(row))

In [44]:
df_directory = '../../data/tidy_data/extod_edu/'
exdi.to_csv(df_directory+'exercise.csv', index=False)
df_total.to_csv(df_directory+'cgm.csv', index=False)
demo_lab.to_csv(df_directory+'demo_lab.csv', index=False)

## Usable bouts

In [45]:
# Convert arrays to sets
set1 = set(df_total.ID.unique())
set2 = set(exdi.ID.unique())

# Find items that are not common in both arrays
unique_to_array1 = set1 - set2
unique_to_array2 = set2 - set1

In [46]:
# Only have
unique_to_array1

{'ext_edu_1030',
 'ext_edu_1032',
 'ext_edu_2015',
 'ext_edu_2016',
 'ext_edu_2034'}

In [47]:
unique_to_array2

{'ext_edu_1009',
 'ext_edu_1016',
 'ext_edu_1017',
 'ext_edu_1018',
 'ext_edu_1023',
 'ext_edu_1028',
 'ext_edu_1033',
 'ext_edu_1037',
 'ext_edu_1041',
 'ext_edu_1043',
 'ext_edu_1044',
 'ext_edu_1050',
 'ext_edu_1052',
 'ext_edu_2014',
 'ext_edu_2021',
 'ext_edu_2022',
 'ext_edu_2033',
 'ext_edu_2044',
 'ext_edu_2047',
 'ext_edu_2048',
 'ext_edu_2049',
 'ext_edu_2050'}

In [48]:
unique_to_array2 = list(unique_to_array2)

In [49]:
filtered_diaries = exdi[~exdi['ID'].isin(unique_to_array2)]


In [50]:
len(filtered_diaries.ID.unique())

67

In [51]:
filtered_diaries.shape[0]

737